<a href="https://colab.research.google.com/github/gabrielcerono/GlioblastomaMultiforme/blob/main/Survival_Analysis_Lammer2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pip
!pip uninstall --yes --quiet osqp
!pip install -U scikit-survival

     |████████████████████████████████| 1.6MB 15.6MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 2.4 MB 15.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
     |████████████████████████████████| 2.3 MB 39.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 212 kB 53.1 MB/s 
  Created wheel for scikit-survival: filename=scikit_survival-0.14.0-cp37-cp37m-linux_x86_64.whl size=4061831 sha256=f304179ce503bb2c6a458af7233cd32fda68967a1532ae4dce4be76206d2f830
  Stored in directory: /root/.cache/pip/wheels/a2/3e/97/3722ba215d3dfe5429c1a7e4f24f535a3f46004fb29a16d505
Successfully built scikit-survival


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk

In [ ]:
pip install eli5

     |████████████████████████████████| 106 kB 15.5 MB/s 


In [ ]:
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.metrics import concordance_index_censored
from sksurv.metrics import brier_score

In [ ]:
dataset = pd.read_excel('/content/glioma.xlsx')

In [ ]:
dataset.head()

,patient ID,sex,age [years],OS [months],"OS status (1=deceased, 0=living)",PFS [months],"progression (1=yes, 0=no)","MGMT methylation (1=methylated, 0=unmethylated)",cHsp70
0,1,m,62,1.2,1,1.2,1,0,high
1,2,m,58,20.9,1,18.2,1,0,high
2,3,f,50,4.0,1,4.6,1,0,low
3,4,m,20,26.0,1,16.1,1,1,high
4,6,m,42,15.8,1,6.2,1,0,low


In [ ]:
dataset = dataset.drop(columns=['patient ID'])

In [ ]:
dataset.columns

Index(['sex', 'age [years]', 'OS [months]', 'OS status (1=deceased, 0=living)',
       'PFS [months]', 'progression (1=yes, 0=no)',
       'MGMT methylation (1=methylated, 0=unmethylated)', 'cHsp70'],
      dtype='object')

In [ ]:
dataset = dataset.rename(columns={'age [years]' : 'age', 'OS [months]' : 'os', 'OS status (1=deceased, 0=living)' : 'death',
       'progression (1=yes, 0=no)' : "didprogress",'MGMT methylation (1=methylated, 0=unmethylated)' : 'mgmt_methylation', 'cHsp70' : 'chsp70_0h1l', 'PFS [months]': 'pfs'})

In [ ]:
dataset.head()

,sex,age,os,death,pfs,didprogress,mgmt_methylation,chsp70_0h1l
0,m,62,1.2,1,1.2,1,0,high
1,m,58,20.9,1,18.2,1,0,high
2,f,50,4.0,1,4.6,1,0,low
3,m,20,26.0,1,16.1,1,1,high
4,m,42,15.8,1,6.2,1,0,low


In [ ]:
from sklearn.preprocessing import LabelEncoder
from pandas import get_dummies

In [ ]:
label_encoder = LabelEncoder()
dataset['sex'] = label_encoder.fit_transform(dataset['sex'])
dataset['chsp70_0h1l'] = label_encoder.fit_transform(dataset['chsp70_0h1l'])

Let's try to predict first overall survival

1.   We need to map death as True or False
2.   We need to put death event first, then the right censored data. Example : (True, 21 months) 
3.   Then we have to pass from pandas DataFrame to structured array
4.   We can recover the data frame from records




In [ ]:
dataset.head(5)

,sex,age,os,death,pfs,didprogress,mgmt_methylation,chsp70_0h1l
0,1,62,1.2,1,1.2,1,0,0
1,1,58,20.9,1,18.2,1,0,0
2,0,50,4.0,1,4.6,1,0,1
3,1,20,26.0,1,16.1,1,1,0
4,1,42,15.8,1,6.2,1,0,1


In [ ]:
dataset.corr(method='pearson')

,sex,age,os,death,pfs,didprogress,mgmt_methylation,chsp70_0h1l
sex,1.000000,-0.269165,-0.016478,0.138013,0.039569,-0.010193,-0.168035,-0.058461
age,-0.269165,1.000000,0.061198,-0.080623,0.177742,-0.161440,0.118375,-0.079750
os,-0.016478,0.061198,1.000000,-0.185224,0.873120,-0.288235,0.330128,-0.313911
death,0.138013,-0.080623,-0.185224,1.000000,-0.223676,0.369274,-0.265897,0.127076
pfs,0.039569,0.177742,0.873120,-0.223676,1.000000,-0.456801,0.320183,-0.328388
didprogress,-0.010193,-0.161440,-0.288235,0.369274,-0.456801,1.000000,-0.010336,0.229416
mgmt_methylation,-0.168035,0.118375,0.330128,-0.265897,0.320183,-0.010336,1.000000,-0.386500
chsp70_0h1l,-0.058461,-0.079750,-0.313911,0.127076,-0.328388,0.229416,-0.386500,1.000000


In [ ]:
Y = dataset[['os', 'death']]

In [ ]:
Y['death'] = Y['death'].map({1: True, 0: False})

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
Y = Y[['death', 'os']]

In [ ]:
Y = Y.to_records(index=False)

In [ ]:
X = dataset.drop(columns = ['death', 'os'])

In [ ]:
from sksurv.ensemble import RandomSurvivalForest
from sklearn.model_selection import train_test_split


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.25)

In [ ]:
rsf = RandomSurvivalForest()
rsf.fit(X_train, y_train)


RandomSurvivalForest(bootstrap=True, max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None, min_samples_leaf=3,
                     min_samples_split=6, min_weight_fraction_leaf=0.0,
                     n_estimators=100, n_jobs=None, oob_score=False,
                     random_state=None, verbose=0, warm_start=False)

In [ ]:
rsf.score(X_test, y_test)

0.8771929824561403

In [ ]:
from eli5.sklearn import PermutationImportance
import eli5


In [ ]:
permuter = PermutationImportance(
    estimator = rsf,
    n_iter = 500,
    random_state = 42
)

In [ ]:
permuter.fit(X_test, y_test)

PermutationImportance(cv='prefit',
                      estimator=RandomSurvivalForest(bootstrap=True,
                                                     max_depth=None,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     max_samples=None,
                                                     min_samples_leaf=3,
                                                     min_samples_split=6,
                                                     min_weight_fraction_leaf=0.0,
                                                     n_estimators=100,
                                                     n_jobs=None,
                                                     oob_score=False,
                                                     random_state=None,
                                                     verbose=0,
                                   

In [ ]:
columns = X_test.columns.to_list()

In [ ]:
feature_importance = permuter.feature_importances_

In [ ]:
feature_importance

array([-0.04701754,  0.04421053,  0.27184211,  0.0022807 ,  0.03649123,
        0.01912281])

In [ ]:
pd.Series(feature_importance, columns).sort_values(ascending = False)

pfs                 0.271842
age                 0.044211
mgmt_methylation    0.036491
chsp70_0h1l         0.019123
didprogress         0.002281
sex                -0.047018
dtype: float64

In [ ]:
metric = eli5.show_weights(
    estimator = permuter,
    top = None, 
    feature_names = columns
)

In [ ]:
metric

Weight,Feature
0.2718 ± 0.2369,pfs
0.0442 ± 0.0926,age
0.0365 ± 0.0793,mgmt_methylation
0.0191 ± 0.0351,chsp70_0h1l
0.0023 ± 0.0284,didprogress
-0.0470 ± 0.0368,sex


In [ ]:
from sksurv.metrics import (concordance_index_censored,
                            concordance_index_ipcw,
                            cumulative_dynamic_auc)

Let's build a function

In [ ]:
def permutation(X, y, model):

  #Let's do a splitting
  X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.25)
  #Let's train the model on the 
  model.fit(X_train, y_train)
  #Let's define the permutation.
  permuter = PermutationImportance(
    estimator = rsf,
    n_iter = 20,
    random_state = 42)
  
  permuter.fit(X_test, y_test)
  
  columns = X_test.columns.to_list()
  feature_importance = permuter.feature_importances_

  metric = eli5.show_weights(
    estimator = permuter,
    top = None, 
    feature_names = columns
  )
  return metric

In [ ]:
permutation(X, Y, rsf)

Weight,Feature
0.1849 ± 0.2046,pfs
0.0176 ± 0.0864,age
0.0015 ± 0.0258,didprogress
-0.0016 ± 0.0341,sex
-0.0478 ± 0.0488,chsp70_0h1l
-0.0655 ± 0.1130,mgmt_methylation


In [ ]:
def permutation(X, y, model, loops):
  rankings = np.zeros(len(X.columns),)
  c_score_total = []

  for x in range(loops):
    
  #Let's do a splitting
    X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.25)
  #Let's train the model on the 
    model.fit(X_train, y_train)
  #Let's define the permutation.
    permuter = PermutationImportance(
    estimator = model,
    n_iter = 10)
  #Let's fit the permutator on the Y test, (This permutation only shuffle the Y test.)
    permuter.fit(X_test, y_test)
  
    columns = X_test.columns.to_list()
  #The feature importance will give us and n array of how much 
    feature_importance = permuter.feature_importances_
    rankings = np.add(feature_importance, rankings)
  #Let's compute the scores
    c_score = model.score(X_test, y_test)
    c_score_total.append(c_score)

  
    
  #Dividing to get the average 
  ranking_terminal = np.true_divide(rankings, loops) 
  c_rank = pd.Series(ranking_terminal, columns).sort_values(ascending = False)
  c_mean = np.mean(np.array(c_score_total))
  return c_rank, c_mean

In [ ]:
rank, score = permutation(X, Y, rsf, 100)

In [ ]:
rank

pfs                 0.227283
mgmt_methylation    0.013246
age                 0.011976
chsp70_0h1l         0.008141
didprogress         0.002269
sex                -0.004871
dtype: float64

In [ ]:
score

0.7735861444368973